In [1]:
from pyspark.sql import SparkSession
import requests

In [2]:
spark = SparkSession.builder \
    .appName("SCD") \
    .config("spark.jars", "/spark-data/scd/libs/snowflake-jdbc-3.13.14.jar,/spark-data/scd/libs/spark-snowflake_2.12-2.16.0-spark_3.4.jar") \
    .getOrCreate()

24/10/23 11:23:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [4]:
url = "https://jsonplaceholder.typicode.com/posts"

In [5]:
response = requests.get(url)

In [6]:
if response.status_code == 200:
    data = response.json()

In [7]:
df = spark.createDataFrame(data)

In [8]:
df.show()

+--------------------+---+--------------------+------+
|                body| id|               title|userId|
+--------------------+---+--------------------+------+
|quia et suscipit\...|  1|sunt aut facere r...|     1|
|est rerum tempore...|  2|        qui est esse|     1|
|et iusto sed quo ...|  3|ea molestias quas...|     1|
|ullam et saepe re...|  4|eum et est occaecati|     1|
|repudiandae venia...|  5|  nesciunt quas odio|     1|
|ut aspernatur cor...|  6|dolorem eum magni...|     1|
|dolore placeat qu...|  7|magnam facilis autem|     1|
|dignissimos aperi...|  8|dolorem dolore es...|     1|
|consectetur animi...|  9|nesciunt iure omn...|     1|
|quo et expedita m...| 10|optio molestias i...|     1|
|delectus reiciend...| 11|et ea vero quia l...|     2|
|itaque id aut mag...| 12|in quibusdam temp...|     2|
|aut dicta possimu...| 13|dolorum ut in vol...|     2|
|fuga et accusamus...| 14|voluptatem eligen...|     2|
|reprehenderit quo...| 15|eveniet quod temp...|     2|
|suscipit 

In [9]:
sfOptions = {
    "sfURL": "me62832.ap-southeast-1.snowflakecomputing.com",  
    "sfUser": "KIMJONG",                                       
    "sfPassword": "Gaurav@123",                                
    "sfDatabase": "SCD_DATABASE",                              
    "sfSchema": "SCD_SCHEMA",                                  
    "sfWarehouse": "SCD_WAREHOUSE",                            
    "sfRole": "ACCOUNTADMIN"                                   
}

In [10]:
df.write \
    .format("snowflake").options(**sfOptions).option("dbtable", "SCD_TABLE").mode("overwrite").save()

24/10/23 11:23:16 WARN SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.5.2 with a connector designed to support Spark 3.4. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.
24/10/23 11:23:16 WARN ServerConnection$: JDBC 3.13.14 is being used. But the certified JDBC version 3.16.1 is recommended.
24/10/23 11:23:26 WARN SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.5.2 with a connector designed to support Spark 3.4. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.


In [11]:
# Reading the table from Snowflake into a PySpark DataFrame
df_snowflake = spark.read \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "SCD_TABLE") \
    .load()

24/10/23 11:23:56 WARN SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.5.2 with a connector designed to support Spark 3.4. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.


In [12]:
# Show the data retrieved from Snowflake
df_snowflake.show()

+--------------------+---+--------------------+------+
|                BODY| ID|               TITLE|USERID|
+--------------------+---+--------------------+------+
|consectetur animi...|  9|nesciunt iure omn...|     1|
|quo et expedita m...| 10|optio molestias i...|     1|
|delectus reiciend...| 11|et ea vero quia l...|     2|
|itaque id aut mag...| 12|in quibusdam temp...|     2|
|aut dicta possimu...| 13|dolorum ut in vol...|     2|
|fuga et accusamus...| 14|voluptatem eligen...|     2|
|reprehenderit quo...| 15|eveniet quod temp...|     2|
|suscipit nam nisi...| 16|sint suscipit per...|     2|
|repellat aut aper...| 89|sint soluta et ve...|     9|
|minus omnis solut...| 90|ad iusto omnis od...|     9|
|libero voluptate ...| 91|        aut amet sed|    10|
|aut et excepturi ...| 92|ratione ex tenetu...|    10|
|dolorem quibusdam...| 93|beatae soluta rec...|    10|
|aspernatur expedi...| 94|qui qui voluptate...|    10|
|earum voluptatem ...| 95|id minus libero i...|    10|
|in non od